# **Data Collection & Cleaning Notebook**

## Objectives

* This notebook will be used to fetch Kaggle datasets and also a BitCoin price dataset from CoinCodex.  Permission was received from Leo Daris, a content manager at CoinCodex.
* The datasets will be merged into one dataset and the data will be inspected and cleaned as necessary

## Inputs

* Kaggle JSON file - the authentication token.
* CoinCodex .csv download - BitCoin prices - https://coincodex.com/crypto/bitcoin/
* Kaggle NASDAQ dataset by Sai Karthik
* Kaggle US Economic Vital Signs: 25 Years Of Macro Data dataset by Eswaran Muthu

## Outputs

* Generate Dataset: outputs/datasets/collection/


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Project5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/Project5'

# Fetch Kaggle Datasets

Install Kaggle package to fetch data

In [4]:
%pip install kaggle==1.5.12

  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=4ea163f8e695b64978b2a6086d00c65789d10af926adcf99301a496b655fad0d
  Stored in directory: /home/cistudent/.cache/pip/wheels/f5/69/4d/d701fc604b9fb09be59718b4056fd5556a22588ce1f25dd090
Successfully built kaggle

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Recognition of token

In [5]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Download Kaggle Datasets

In [ ]:
dataset_1 = "sai14karthik/nasdq-dataset"
dataset_2 = "eswaranmuthu/u-s-economic-vital-signs-25-years-of-macro-data"
DestinationFolder = "inputs/datasets/raw"

!kaggle datasets download -d {dataset_1} -p {DestinationFolder}
!kaggle datasets download -d {dataset_2} -p {DestinationFolder}

100%|█████████████████████████████████████████| 126k/126k [00:00<00:00, 514kB/s]
100%|█████████████████████████████████████████| 126k/126k [00:00<00:00, 513kB/s]
  0%|                                               | 0.00/8.65k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 8.65k/8.65k [00:00<00:00, 24.8MB/s]


Unzip the files

In [11]:
import os
import zipfile

for file in os.listdir(DestinationFolder):
    if file.endswith(".zip"):
        zip_path = os.path.join(DestinationFolder, file)
        print(f"Unzipping: {zip_path}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DestinationFolder)
        os.remove(zip_path)


Unzipping: inputs/datasets/raw/nasdq-dataset.zip
Unzipping: inputs/datasets/raw/u-s-economic-vital-signs-25-years-of-macro-data.zip


Load and inspect the data

In [4]:
import pandas as pd
df_bitcoin = pd.read_csv(f"inputs/datasets/raw/bitcoin_2010-07-17_2025-07-31.csv")
df_bitcoin.head()

,Start,End,Open,High,Low,Close,Volume,Market Cap
0,2025-07-31,2025-08-01,117823.0,118867.0,115606.0,115606.0,6.462546e+10,2.347673e+12
1,2025-07-30,2025-07-31,117796.0,118699.0,116027.0,117800.0,5.748423e+10,2.343903e+12
2,2025-07-29,2025-07-30,118100.0,119095.0,117084.0,117877.0,6.025039e+10,2.351844e+12
3,2025-07-28,2025-07-29,119370.0,119759.0,117435.0,117883.0,5.371660e+10,2.360848e+12
4,2025-07-27,2025-07-28,117944.0,119767.0,117842.0,119429.0,3.495047e+10,2.357912e+12


In [5]:
import pandas as pd
df_macro = pd.read_csv(f"inputs/datasets/raw/macro_data_25yrs.csv")
df_macro.head()

,Date,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR
0,2018-04-03,13993.9,2.87,1.69,250.227,2.470996,1.83
1,2018-04-04,13993.9,2.87,1.69,250.227,2.470996,1.74
2,2018-04-05,13993.9,2.87,1.69,250.227,2.470996,1.75
3,2018-04-06,13993.9,2.87,1.69,250.227,2.470996,1.75
4,2018-04-09,13993.9,2.87,1.69,250.227,2.470996,1.75


In [6]:
import pandas as pd
df_nasdaq = pd.read_csv(f"inputs/datasets/raw/nasdq.csv")
df_nasdaq.head()

,Date,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil
0,2010-01-04,6.640000,6.810000,6.633333,6.746667,6514500.0,0.11,1.4419,20.04,0.17,0.12,1117.699951,81.510002
1,2010-01-05,6.643333,6.773333,6.643333,6.766667,4445100.0,0.11,1.4402,19.35,0.18,0.12,1118.099976,81.769997
2,2010-01-06,6.733333,6.786667,6.720000,6.763333,7340100.0,0.11,1.4404,19.16,0.19,0.12,1135.900024,83.180000
3,2010-01-07,6.750000,6.766667,6.630000,6.673333,8498400.0,0.11,1.4314,19.06,0.20,0.10,1133.099976,82.660004
4,2010-01-08,6.676667,6.766667,6.626667,6.743333,4347600.0,0.11,1.4357,18.13,0.20,0.11,1138.199951,82.750000


---

# Data Inspection & Cleaning

Convert all datasets' date using datetime

In [7]:
df_bitcoin['Date'] = pd.to_datetime(df_bitcoin['End'])
df_macro['Date'] = pd.to_datetime(df_macro['Date'])
df_nasdaq['Date'] = pd.to_datetime(df_nasdaq['Date'])

Sort date values

In [8]:
df_bitcoin.sort_values('Date', inplace=True)
df_macro.sort_values('Date', inplace=True)
df_nasdaq.sort_values('Date', inplace=True)

Merge the datasets & rename BitCoin 'Close' column as 'BitCoin_Close' to remove ambiguity across datasets

In [9]:
df_merged = pd.merge(df_nasdaq, df_macro, on='Date', how='inner')

In [10]:
df_bitcoin.rename(columns={'Close': 'BitCoin_Close'}, inplace=True)

In [11]:
df_final = pd.merge(df_merged, df_bitcoin[['Date', 'BitCoin_Close']], on='Date', how='left')

In [12]:
df_final.head()


,Date,Open,High,Low,Close,Volume,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR,BitCoin_Close
0,2018-04-03,28.526667,28.903334,28.366667,28.883333,4917300.0,1.69,1.2261,21.10,0.60,1.69,1332.800049,63.509998,13993.9,2.87,1.69,250.227,2.470996,1.83,7061.622526
1,2018-04-04,28.469999,28.799999,28.379999,28.740000,3822600.0,1.69,1.2292,20.06,0.64,1.69,1335.800049,63.369999,13993.9,2.87,1.69,250.227,2.470996,1.74,7454.691790
2,2018-04-05,28.833332,28.980000,28.670000,28.770000,3174300.0,1.69,1.2230,18.94,0.64,1.69,1324.300049,63.540001,13993.9,2.87,1.69,250.227,2.470996,1.75,6840.936110
3,2018-04-06,28.633333,28.730000,28.163334,28.400000,2808000.0,1.69,1.2274,21.49,0.64,1.69,1331.900024,62.060001,13993.9,2.87,1.69,250.227,2.470996,1.75,6819.726657
4,2018-04-09,28.486668,28.886667,28.320000,28.430000,1798200.0,1.69,1.2320,21.77,0.61,1.69,1336.300049,63.419998,13993.9,2.87,1.69,250.227,2.470996,1.75,7000.923355


In [13]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                1714 non-null   datetime64[ns]
 1   Open                1714 non-null   float64       
 2   High                1714 non-null   float64       
 3   Low                 1714 non-null   float64       
 4   Close               1714 non-null   float64       
 5   Volume              1714 non-null   float64       
 6   InterestRate        1714 non-null   float64       
 7   ExchangeRate        1714 non-null   float64       
 8   VIX                 1714 non-null   float64       
 9   TEDSpread           1714 non-null   float64       
 10  EFFR                1714 non-null   float64       
 11  Gold                1714 non-null   float64       
 12  Oil                 1714 non-null   float64       
 13  M2_Money_Supply     1714 non-null   float64     

Check for missing values:

In [14]:
print(df_final.isnull().sum())

Date                   0
Open                   0
High                   0
Low                    0
Close                  0
Volume                 0
InterestRate           0
ExchangeRate           0
VIX                    0
TEDSpread              0
EFFR                   0
Gold                   0
Oil                    0
M2_Money_Supply        0
10Y Treasury Yield     0
Fed Funds Rate         0
CPI                    0
Inflation_Rate_%       0
SOFR                  72
BitCoin_Close          0
dtype: int64


Use interpolation to fill in missing values based on those values around them in the SOFR column

In [15]:
df_final['SOFR'] = df_final['SOFR'].interpolate(method='linear')


In [16]:
print(df_final.isnull().sum())

Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Volume                0
InterestRate          0
ExchangeRate          0
VIX                   0
TEDSpread             0
EFFR                  0
Gold                  0
Oil                   0
M2_Money_Supply       0
10Y Treasury Yield    0
Fed Funds Rate        0
CPI                   0
Inflation_Rate_%      0
SOFR                  0
BitCoin_Close         0
dtype: int64


### Drop unnecessary columns & rename others for clarity

* Drop open, low and high prices as we're only interested in the close prices.  Also drop InterestRate & EFFR as these are duplicated with Fed Funds Rate

* Rename Close and Volume columns for better clarity

In [17]:
df_final.drop(columns=['Open', 'Low', 'High', 'InterestRate', 'EFFR'], inplace=True)
df_final.rename(columns={
    'Close': 'Nasdaq_Close',
    'Volume': 'Nasdaq_Volume',
    }, inplace=True)
df_final.head()

,Date,Nasdaq_Close,Nasdaq_Volume,ExchangeRate,VIX,TEDSpread,Gold,Oil,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR,BitCoin_Close
0,2018-04-03,28.883333,4917300.0,1.2261,21.10,0.60,1332.800049,63.509998,13993.9,2.87,1.69,250.227,2.470996,1.83,7061.622526
1,2018-04-04,28.740000,3822600.0,1.2292,20.06,0.64,1335.800049,63.369999,13993.9,2.87,1.69,250.227,2.470996,1.74,7454.691790
2,2018-04-05,28.770000,3174300.0,1.2230,18.94,0.64,1324.300049,63.540001,13993.9,2.87,1.69,250.227,2.470996,1.75,6840.936110
3,2018-04-06,28.400000,2808000.0,1.2274,21.49,0.64,1331.900024,62.060001,13993.9,2.87,1.69,250.227,2.470996,1.75,6819.726657
4,2018-04-09,28.430000,1798200.0,1.2320,21.77,0.61,1336.300049,63.419998,13993.9,2.87,1.69,250.227,2.470996,1.75,7000.923355


In [18]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                1714 non-null   datetime64[ns]
 1   Nasdaq_Close        1714 non-null   float64       
 2   Nasdaq_Volume       1714 non-null   float64       
 3   ExchangeRate        1714 non-null   float64       
 4   VIX                 1714 non-null   float64       
 5   TEDSpread           1714 non-null   float64       
 6   Gold                1714 non-null   float64       
 7   Oil                 1714 non-null   float64       
 8   M2_Money_Supply     1714 non-null   float64       
 9   10Y Treasury Yield  1714 non-null   float64       
 10  Fed Funds Rate      1714 non-null   float64       
 11  CPI                 1714 non-null   float64       
 12  Inflation_Rate_%    1714 non-null   float64       
 13  SOFR                1714 non-null   float64     

### Data Cleaning Summary

* BitCoin's 'Close' column was renamed to 'BitCoin_Close'
* Interpolation was used to handle missing data evident on the SOFR column
* Unnecessary columns were removed to avoid duplicated columns in the final dataset
* Nasdaq 'Close' and 'Volume' columns had 'Nasdaq' included in the column names for futher clarity

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [19]:
import os
try:
  os.makedirs(name='outputs/datasets/collection')
except Exception as e:
  print(e)

df_final.to_csv(f"outputs/datasets/collection/BitCoinVsMacroNasdaq_v2.csv",index=False)

[Errno 17] File exists: 'outputs/datasets/collection'
